## Import Library
Kita mulai dengan mengimpor semua library yang diperlukan untuk memproses dan menggabungkan data:
- `os` & `glob`: untuk membaca file dari folder
- `pandas` & `numpy`: untuk manipulasi data
- `statistics.mode`: untuk menghitung modus (nilai paling sering muncul)

In [12]:
# Import library yang dibutuhkan
import os
import pandas as pd
import numpy as np
import glob
from statistics import mode

## Menggabungkan Semua File CO2
Semua file CSV dalam folder `dataset/data-raw/co2` akan digabung menjadi satu dataframe. Kemudian kita hanya ambil kolom `timestamp` dan `co2`, serta pastikan kedua kolom memiliki format data yang benar.

In [13]:
# Tentukan path ke folder data CO2
folder_path_co2 = 'dataset/data-raw/co2'
all_csv_files = glob.glob(os.path.join(folder_path_co2, "*.csv"))

# Gabungkan semua file CSV
df_co2 = pd.concat([pd.read_csv(f) for f in all_csv_files], ignore_index=True)
df_co2 = df_co2[['timestamp', 'co2']]

# Konversi tipe data
df_co2['timestamp'] = pd.to_datetime(df_co2['timestamp'], errors='coerce')
df_co2['co2'] = pd.to_numeric(df_co2['co2'], errors='coerce')

## Sampling Data CO2 Per Menit
Data CO2 yang awalnya tercatat beberapa kali per menit akan diubah menjadi satu titik data per menit. Nilai yang dipilih adalah **modus**, yaitu nilai yang paling sering muncul setiap menitnya.

In [14]:
# Bulatkan waktu ke menit terdekat
df_co2['minute'] = df_co2['timestamp'].dt.floor('min')

# Sampling modus CO2 per menit
minute_co2 = df_co2.groupby('minute')['co2'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
).reset_index()

# Lengkapi semua menit agar tidak ada yang hilang
full_minutes = pd.date_range(
    start=df_co2['minute'].min().floor('D'),
    end=df_co2['minute'].max().ceil('D') - pd.Timedelta(minutes=1),
    freq='1min'
)
full_minutes_df = pd.DataFrame({'minute': full_minutes})
co2_sampled = full_minutes_df.merge(minute_co2, on='minute', how='left')

## Menyimpan Data CO2 yang Telah Disampling
Dataset hasil sampling disimpan di folder `dataset/data-clean/` dengan nama `co2_per_minute.csv`.

In [15]:
# Simpan hasil sampling CO2
output_co2_path = 'dataset/data-clean/co2_per_minute.csv'
co2_sampled.to_csv(output_co2_path, index=False)
print(f"CO2 per menit disimpan di: {output_co2_path}")

CO2 per menit disimpan di: dataset/data-clean/co2_per_minute.csv


## Menggabungkan dan Membersihkan Data Cuaca
Semua file cuaca digabung dan kolom yang tidak relevan seperti `direction`, `angle`, dan `wind_speed` dihapus. Lalu kolom `timestamp` dikonversi menjadi format datetime.

In [16]:
folder_path_weather = 'dataset/data-raw/cuaca'
file_list_weather = sorted(os.listdir(folder_path_weather))

df_list_weather = []
for file in file_list_weather:
    if file.endswith('.csv'):
        df = pd.read_csv(os.path.join(folder_path_weather, file))
        df_list_weather.append(df)

df_weather = pd.concat(df_list_weather, ignore_index=True)
df_weather = df_weather.drop(['direction', 'angle', 'wind_speed'], axis=1)  # Hapus kolom tak diperlukan
df_weather['timestamp'] = pd.to_datetime(df_weather['timestamp'])

## Sampling Data Cuaca Per Menit
Data cuaca juga disampling agar memiliki satu titik data per menit dalam rentang 24 April–7 Mei 2025. Jika data tidak ada, akan diisi sebagai `NaN`.

In [17]:
start_date = '2025-04-24'
end_date = '2025-05-07'

# Filter tanggal
filtered_weather = df_weather[(df_weather['timestamp'] >= start_date) & (df_weather['timestamp'] <= end_date)].copy()

# Lengkapi waktu setiap menit
full_range = pd.date_range(start=start_date + ' 00:00:00', end=end_date + ' 23:59:00', freq='min')
full_weather_df = pd.DataFrame({'timestamp': full_range})

weather_sampled = pd.merge(full_weather_df, filtered_weather, on='timestamp', how='left')

## Menyimpan Data Cuaca yang Telah Disampling
Data cuaca yang sudah terisi lengkap per menit disimpan di `dataset/data-clean/cuaca_per_menit.csv`.

In [18]:
output_weather_path = 'dataset/data-clean/cuaca_per_menit.csv'
weather_sampled.to_csv(output_weather_path, index=False)
print(f"Cuaca per menit disimpan di: {output_weather_path}")

Cuaca per menit disimpan di: dataset/data-clean/cuaca_per_menit.csv


## Menggabungkan Data CO2 dan Cuaca
Setelah kedua data sudah disampling per menit, kini saatnya menggabungkannya berdasarkan waktu (`timestamp`). Kita pastikan tidak ada duplikasi timestamp, dan semua nilai sudah rapi.

In [19]:
# Load ulang data yang telah disampling
co2_df = pd.read_csv('dataset/data-clean/co2_per_minute.csv')
cuaca_df = pd.read_csv('dataset/data-clean/cuaca_per_menit.csv')

# Konversi waktu agar bisa digabung
co2_df['minute'] = pd.to_datetime(co2_df['minute'])
cuaca_df['timestamp'] = pd.to_datetime(cuaca_df['timestamp'])

# Gabungkan berdasarkan waktu
merged_df = pd.merge(co2_df, cuaca_df, left_on='minute', right_on='timestamp', how='left')

# Finalisasi
merged_df.drop(columns=['timestamp'], inplace=True)
merged_df.rename(columns={'minute': 'timestamp'}, inplace=True)

## Menyimpan Data Gabungan
File akhir yang berisi gabungan CO2 dan cuaca per menit disimpan ke dalam `dataset/data-clean/data_output_collecting.csv` dan siap digunakan untuk tahap selanjutnya seperti EDA atau modeling.

In [20]:
output_merged_path = 'dataset/data-clean/data_output_collecting.csv'
merged_df.to_csv(output_merged_path, index=False)
print(f"Data gabungan berhasil disimpan di: {output_merged_path}")

Data gabungan berhasil disimpan di: dataset/data-clean/data_output_collecting.csv
